In [ ]:
import json
with open("annotations/train.json") as f:
    data = json.load(f)
for i in range(100):
    print(data["images"][-i-1]) 

{'license': 0, 'url': None, 'file_name': 'P0137_98.jpg', 'height': 800, 'width': 800, 'date_captured': None, 'id': 3641}
{'license': 0, 'url': None, 'file_name': 'P0137_93.jpg', 'height': 800, 'width': 800, 'date_captured': None, 'id': 3640}
{'license': 0, 'url': None, 'file_name': 'P0137_90.jpg', 'height': 800, 'width': 800, 'date_captured': None, 'id': 3639}
{'license': 0, 'url': None, 'file_name': 'P0137_89.jpg', 'height': 800, 'width': 800, 'date_captured': None, 'id': 3638}
{'license': 0, 'url': None, 'file_name': 'P0137_88.jpg', 'height': 800, 'width': 800, 'date_captured': None, 'id': 3637}
{'license': 0, 'url': None, 'file_name': 'P0137_87.jpg', 'height': 800, 'width': 800, 'date_captured': None, 'id': 3636}
{'license': 0, 'url': None, 'file_name': 'P0137_86.jpg', 'height': 800, 'width': 800, 'date_captured': None, 'id': 3635}
{'license': 0, 'url': None, 'file_name': 'P0137_85.jpg', 'height': 800, 'width': 800, 'date_captured': None, 'id': 3634}
{'license': 0, 'url': None, 'fil

In [ ]:
import os
import json
import shutil

def load_json_file(file_path):
    # 读取 JSON 文件，注意编码方式可能需要根据实际情况调整
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def get_image_list(json_data):
    """
    从 JSON 数据中提取图片名称列表：
    - 如果是 COCO 格式（图片信息存放在'images'字段中），提取每个图片字典中的'file_name'字段；
    - 如果 JSON 文件直接为图片名称列表，则直接返回该列表；
    - 其他格式则返回空列表。
    """
    if isinstance(json_data, dict) and 'images' in json_data:
        return [img['file_name'] for img in json_data['images']]
    elif isinstance(json_data, list):
        return json_data
    else:
        return []

def main():
    # 原始图片目录和json文件所在目录
    jpeg_dir = 'JPEGImages'
    annotations_dir = 'annotations'
    
    # 目标目录
    train_dir = 'train'
    val_dir = 'val'
    
    # 创建目标目录（如果不存在）
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    
    # 加载 train.json 和 val.json
    train_json_path = os.path.join(annotations_dir, 'train.json')
    val_json_path = os.path.join(annotations_dir, 'test_val.json')
    
    train_data = load_json_file(train_json_path)
    val_data = load_json_file(val_json_path)
    
    train_images = get_image_list(train_data)
    val_images = get_image_list(val_data)
    
    # 将训练集图片从 JPEGImages 移动到 train 目录
    for image_name in train_images:
        src = os.path.join(jpeg_dir, image_name)
        dst = os.path.join(train_dir, image_name)
        if os.path.exists(src):
            shutil.move(src, dst)
            print(f"移动 {src} 到 {dst}")
        else:
            print(f"未找到文件: {src}")
    
    # 将验证集图片从 JPEGImages 移动到 val 目录
    for image_name in val_images:
        src = os.path.join(jpeg_dir, image_name)
        dst = os.path.join(val_dir, image_name)
        if os.path.exists(src):
            shutil.move(src, dst)
            print(f"移动 {src} 到 {dst}")
        else:
            print(f"未找到文件: {src}")

if __name__ == '__main__':
    main()


移动 JPEGImages\P0001_0_800_7200_8000.jpg 到 train\P0001_0_800_7200_8000.jpg
移动 JPEGImages\P0001_0_800_8400_9200.jpg 到 train\P0001_0_800_8400_9200.jpg
移动 JPEGImages\P0001_0_800_9000_9800.jpg 到 train\P0001_0_800_9000_9800.jpg
移动 JPEGImages\P0001_0_800_9600_10400.jpg 到 train\P0001_0_800_9600_10400.jpg
移动 JPEGImages\P0001_1200_2000_0_800.jpg 到 train\P0001_1200_2000_0_800.jpg
移动 JPEGImages\P0001_1200_2000_4200_5000.jpg 到 train\P0001_1200_2000_4200_5000.jpg
移动 JPEGImages\P0001_1200_2000_7800_8600.jpg 到 train\P0001_1200_2000_7800_8600.jpg
移动 JPEGImages\P0001_1200_2000_8400_9200.jpg 到 train\P0001_1200_2000_8400_9200.jpg
移动 JPEGImages\P0001_1200_2000_9600_10400.jpg 到 train\P0001_1200_2000_9600_10400.jpg
移动 JPEGImages\P0001_1800_2600_10190_10990.jpg 到 train\P0001_1800_2600_10190_10990.jpg
移动 JPEGImages\P0001_1800_2600_2400_3200.jpg 到 train\P0001_1800_2600_2400_3200.jpg
移动 JPEGImages\P0001_1800_2600_3000_3800.jpg 到 train\P0001_1800_2600_3000_3800.jpg
移动 JPEGImages\P0001_1800_2600_3600_4400.jpg 到 tr

In [ ]:
import os
import glob
import json
import xml.etree.ElementTree as ET
import argparse

def parse_xml(xml_file, class_to_id):
    """
    解析单个 XML 文件，返回图像信息及该图像中的所有目标标注
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    # 解析图像尺寸
    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)
    
    # 构造图像信息，这里假设图像文件名与 XML 文件名相同，只是后缀为 .jpg
    image_info = {
        "width": width,
        "height": height,
        "file_name": os.path.splitext(os.path.basename(xml_file))[0] + ".jpg"
    }
    
    annotations = []
    # 遍历图像中的所有目标
    for obj in root.findall('object'):
        cls = obj.find('name').text
        if cls not in class_to_id:
            print(f"警告: 类别 {cls} 不在预定义类别中，跳过该目标。")
            continue
        category_id = class_to_id[cls]
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        bbox_width = xmax - xmin
        bbox_height = ymax - ymin
        
        annotation = {
            "bbox": [xmin, ymin, bbox_width, bbox_height],
            "area": bbox_width * bbox_height,
            "category_id": category_id,
            "iscrowd": 0,
        }
        annotations.append(annotation)
    return image_info, annotations

def convert_annotations(input_dir, output_json):
    """
    遍历 input_dir 下所有 XML 文件，将标注转换为 COCO 格式 JSON 并保存到 output_json 中
    """
    # 定义类别及其 id
    classes = ('Boeing737', 'Boeing787', 'A330', 'A220', 'A320/321', 'ARJ21', 'other')
    class_to_id = {cls: idx + 1 for idx, cls in enumerate(classes)}
    
    coco_output = {
        "images": [],
        "annotations": [],
        "categories": []
    }
    
    # 添加类别信息
    for cls, cls_id in class_to_id.items():
        coco_output["categories"].append({
            "id": cls_id,
            "name": cls
        })
        
    annotation_id = 1
    image_id = 1
    
    xml_files = glob.glob(os.path.join(input_dir, "*.xml"))
    for xml_file in xml_files:
        image_info, annots = parse_xml(xml_file, class_to_id)
        image_info["id"] = image_id
        coco_output["images"].append(image_info)
        
        for annot in annots:
            annot["id"] = annotation_id
            annot["image_id"] = image_id
            coco_output["annotations"].append(annot)
            annotation_id += 1
        
        image_id += 1
    
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(coco_output, f, indent=4, ensure_ascii=False)
    print(f"COCO 标注已保存到 {output_json}")

convert_annotations("SAR-AIRcraft-1.0/Annotations", "SAR-AIRcraft-1.0/train_val_test.json")

COCO 标注已保存到 SAR-AIRcraft-1.0/train.json


In [7]:
import json
import random
import argparse

def split_coco(input_json, train_json, val_json, test_json, train_ratio=0.7, val_ratio=0.1, test_ratio=0.2):
    # 读取 COCO 格式标注文件
    with open(input_json, "r", encoding="utf-8") as f:
        coco_data = json.load(f)
    
    images = coco_data["images"]
    annotations = coco_data["annotations"]
    categories = coco_data["categories"]

    # 读取 train.txt, val.txt, test.txt
    with open("train.txt", "r") as f:
        train_filenames = [line.strip() + '.jpg' for line in f]
        train_images = [img for img in images if img["file_name"] in train_filenames]
    with open("val.txt", "r") as f:
        val_filenames = [line.strip() + '.jpg' for line in f]
        val_images = [img for img in images if img["file_name"] in val_filenames]
    with open("test.txt", "r") as f:
        test_filenames = [line.strip() + '.jpg' for line in f]
        test_images = [img for img in images if img["file_name"] in test_filenames]
    
    # 根据图像 id 进行划分
    train_ids = set(img["id"] for img in train_images)
    val_ids = set(img["id"] for img in val_images)
    test_ids = set(img["id"] for img in test_images)

    print(f"训练集数量：{len(train_ids)}，验证集数量：{len(val_ids)}，测试集数量：{len(test_ids)}")

    train_annotations = [ann for ann in annotations if ann["image_id"] in train_ids]
    val_annotations = [ann for ann in annotations if ann["image_id"] in val_ids]
    test_annotations = [ann for ann in annotations if ann["image_id"] in test_ids]

    print(f"训练集标注数量：{len(train_annotations)}，验证集标注数量：{len(val_annotations)}，测试集标注数量：{len(test_annotations)}")

    # 按照 id 顺序排序
    train_images.sort(key=lambda x: x["id"])
    train_annotations.sort(key=lambda x: x["id"])
    val_images.sort(key=lambda x: x["id"])
    val_annotations.sort(key=lambda x: x["id"])
    test_images.sort(key=lambda x: x["id"])
    test_annotations.sort(key=lambda x: x["id"])

    # 构造三个子集的 COCO 格式数据
    coco_train = {
        "images": train_images,
        "annotations": train_annotations,
        "categories": categories
    }
    coco_val = {
        "images": val_images,
        "annotations": val_annotations,
        "categories": categories
    }
    coco_test = {
        "images": test_images,
        "annotations": test_annotations,
        "categories": categories
    }

    # 分别写入 JSON 文件
    with open(train_json, "w", encoding="utf-8") as f:
        json.dump(coco_train, f, indent=4, ensure_ascii=False)
    with open(val_json, "w", encoding="utf-8") as f:
        json.dump(coco_val, f, indent=4, ensure_ascii=False)
    with open(test_json, "w", encoding="utf-8") as f:
        json.dump(coco_test, f, indent=4, ensure_ascii=False)

    print(f"划分完成！生成文件：\n训练集：{train_json}，验证集：{val_json}，测试集：{test_json}")

split_coco("browse_dataset.json", "train.json", "val.json", "test.json")


训练集数量：3047，验证集数量：442，测试集数量：879
训练集标注数量：11551，验证集标注数量：1962，测试集标注数量：2950
划分完成！生成文件：
训练集：train.json，验证集：val.json，测试集：test.json


In [1]:
import os
import json
import shutil

def load_json_file(file_path):
    # 读取 JSON 文件，注意编码方式可能需要根据实际情况调整
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def get_image_list(json_data):
    """
    从 JSON 数据中提取图片名称列表：
    - 如果是 COCO 格式（图片信息存放在'images'字段中），提取每个图片字典中的'file_name'字段；
    - 如果 JSON 文件直接为图片名称列表，则直接返回该列表；
    - 其他格式则返回空列表。
    """
    if isinstance(json_data, dict) and 'images' in json_data:
        return [img['file_name'] for img in json_data['images']]
    elif isinstance(json_data, list):
        return json_data
    else:
        return []

def main():
    # 原始图片目录和json文件所在目录
    jpeg_dir = 'Images'
    annotations_dir = 'annotations'
    
    # 目标目录
    train_dir = 'train'
    val_dir = 'val'
    test_dir = 'test'
    
    # 创建目标目录（如果不存在）
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    
    # 加载 train.json 和 val.json
    train_json_path = os.path.join(annotations_dir, 'train.json')
    val_json_path = os.path.join(annotations_dir, 'val.json')
    test_json_path = os.path.join(annotations_dir, 'test.json')
    
    train_data = load_json_file(train_json_path)
    val_data = load_json_file(val_json_path)
    test_data = load_json_file(test_json_path)
    
    train_images = get_image_list(train_data)
    val_images = get_image_list(val_data)
    test_images = get_image_list(test_data)
    
    # 将训练集图片从 JPEGImages 移动到 train 目录
    for image_name in train_images:
        src = os.path.join(jpeg_dir, image_name)
        dst = os.path.join(train_dir, image_name)
        if os.path.exists(src):
            shutil.move(src, dst)
            print(f"移动 {src} 到 {dst}")
        else:
            print(f"未找到文件: {src}")
    
    # 将验证集图片从 JPEGImages 移动到 val 目录
    for image_name in val_images:
        src = os.path.join(jpeg_dir, image_name)
        dst = os.path.join(val_dir, image_name)
        if os.path.exists(src):
            shutil.move(src, dst)
            print(f"移动 {src} 到 {dst}")
        else:
            print(f"未找到文件: {src}")

    # 将测试集图片从 JPEGImages 移动到 test 目录
    for image_name in test_images:
        src = os.path.join(jpeg_dir, image_name)
        dst = os.path.join(test_dir, image_name)
        if os.path.exists(src):
            shutil.move(src, dst)
            print(f"移动 {src} 到 {dst}")
        else:
            print(f"未找到文件: {src}")

main()


移动 Images\0000001.jpg 到 train\0000001.jpg
移动 Images\0000002.jpg 到 train\0000002.jpg
移动 Images\0000003.jpg 到 train\0000003.jpg
移动 Images\0000004.jpg 到 train\0000004.jpg
移动 Images\0000005.jpg 到 train\0000005.jpg
移动 Images\0000006.jpg 到 train\0000006.jpg
移动 Images\0000007.jpg 到 train\0000007.jpg
移动 Images\0000008.jpg 到 train\0000008.jpg
移动 Images\0000010.jpg 到 train\0000010.jpg
移动 Images\0000012.jpg 到 train\0000012.jpg
移动 Images\0000015.jpg 到 train\0000015.jpg
移动 Images\0000017.jpg 到 train\0000017.jpg
移动 Images\0000018.jpg 到 train\0000018.jpg
移动 Images\0000019.jpg 到 train\0000019.jpg
移动 Images\0000021.jpg 到 train\0000021.jpg
移动 Images\0000022.jpg 到 train\0000022.jpg
移动 Images\0000024.jpg 到 train\0000024.jpg
移动 Images\0000025.jpg 到 train\0000025.jpg
移动 Images\0000026.jpg 到 train\0000026.jpg
移动 Images\0000027.jpg 到 train\0000027.jpg
移动 Images\0000028.jpg 到 train\0000028.jpg
移动 Images\0000029.jpg 到 train\0000029.jpg
移动 Images\0000031.jpg 到 train\0000031.jpg
移动 Images\0000032.jpg 到 train\0000

In [2]:
import json
import os
import matplotlib.pyplot as plt
import numpy as np

# 加载完整数据集
def load_dataset(json_path):
    with open(json_path, 'r') as f:
        return json.load(f)

# 计算统计信息
def compute_statistics(dataset):
    # 图像数量
    image_count = len(dataset['images'])
    
    # 目标数量
    annotation_count = len(dataset['annotations'])
    
    # 每个类别的目标数量
    categories = {cat['id']: cat['name'] for cat in dataset['categories']}
    category_counts = {}
    for ann in dataset['annotations']:
        cat_id = ann['category_id']
        if cat_id not in category_counts:
            category_counts[cat_id] = 0
        category_counts[cat_id] += 1
    
    # 每张图像的尺寸分布
    size_distribution = {}
    for img in dataset['images']:
        size = f"{img['width']}x{img['height']}"
        if size not in size_distribution:
            size_distribution[size] = 0
        size_distribution[size] += 1
    
    return {
        'image_count': image_count,
        'annotation_count': annotation_count,
        'category_counts': category_counts,
        'categories': categories,
        'size_distribution': size_distribution
    }

# 调用统计函数分析训练集、验证集和测试集
train_stats = compute_statistics(load_dataset('train.json'))
val_stats = compute_statistics(load_dataset('val.json'))
test_stats = compute_statistics(load_dataset('test.json')) if os.path.exists('test.json') else None

print("训练集统计信息：")
print(train_stats)
print("验证集统计信息：")
print(val_stats)
print("测试集统计信息：")
print(test_stats)

训练集统计信息：
{'image_count': 3047, 'annotation_count': 11551, 'category_counts': {2: 1930, 1: 1755, 4: 3023, 3: 251, 7: 2508, 6: 447, 5: 1637}, 'categories': {1: 'Boeing737', 2: 'Boeing787', 3: 'A330', 4: 'A220', 5: 'A320/321', 6: 'ARJ21', 7: 'other'}, 'size_distribution': {'1500x1500': 699, '1200x1200': 777, '800x800': 836, '1000x1000': 735}}
验证集统计信息：
{'image_count': 442, 'annotation_count': 1962, 'category_counts': {7: 715, 2: 261, 1: 252, 5: 82, 6: 378, 4: 247, 3: 27}, 'categories': {1: 'Boeing737', 2: 'Boeing787', 3: 'A330', 4: 'A220', 5: 'A320/321', 6: 'ARJ21', 7: 'other'}, 'size_distribution': {'1200x1200': 63, '1500x1500': 93, '800x800': 168, '1000x1000': 118}}
测试集统计信息：
{'image_count': 879, 'annotation_count': 2950, 'category_counts': {1: 550, 4: 460, 2: 454, 6: 362, 7: 1041, 3: 31, 5: 52}, 'categories': {1: 'Boeing737', 2: 'Boeing787', 3: 'A330', 4: 'A220', 5: 'A320/321', 6: 'ARJ21', 7: 'other'}, 'size_distribution': {'800x800': 308, '1200x1200': 198, '1500x1500': 141, '1000x1000':